# From Mantid to Scipp
## Data types
### Workspaces

| Mantid | Scipp |
| ---| --- |
| `Workspace2D` | `DataArray` |
| `EventWorkspace` | `DataArray` |
| `WorkspaceSingleValue` | `DataArray` |
| `MDHistoWorkspace` | `DataArray` |
| `MDEventWorkspace` | - |
| `TableWorkspace` | `Dataset` |
| `WorkspaceGroup` | `Dataset` (aligned dimensions), otherwise Python `list` or `dict` |

#### Notes
- In many cases it may be desirable to use `Dataset` instead of `DataArray`.
- Scipp (other rather conversion to scipp) is currently still incomplete and does not carry over all information from a workspace.

### Other

| Mantid | Scipp |
| ---| --- |
| `DetectorInfo` | `Dataset` |

## Algorithms

### Notes
- In **Mantid** a Python variable referencing **a workspace is** under the hood **a global variable**.
  Unless specific otherwise the variable name is the name of the workspace in the [AnalysisDataService](https://docs.mantidproject.org/nightly/concepts/AnalysisDataService.html).
  For marginally more clarity, the examples in the following therefore use the string-based syntax for specifying output workspaces.
  *In scipp there is no such limitation and everything behaves just like normal variables in Python.*
- Unless stated otherwise, the following code examples assume datasets or data arrays have `Dim.Tof` for what Mantid calls "X" and `Dim.Spectrum` why Mantid calls "Y" or "spectrum axis".

In [ ]:
import mantid.simpleapi as mantid
import scipp as sc
from scipp import Dim
import numpy as np

### Generic algorithms

#### CloneWorkspace

In [ ]:
mantid.CloneWorkspace(InputWorkspace='data', OutputWorkspace='copy')

Equivalent in scipp:

In [ ]:
copy = data.copy()

#### DeleteWorkspace

In [ ]:
mantid.DeleteWorkspace(Workspace='data')

Equivalent in scipp:

In [ ]:
del data

#### ExtractSingleSpectrum

#### Rebin  `Workspace2D` into `Workspace2D`

In [ ]:
mantid.Rebin(InputWorkspace='histo', OutputWorkspace='histo', Params='0,100,20000')

Equivalent in scipp:

In [ ]:
edges = sc.Variable([Dim.Tof], values=np.arange(0.0, 20000.0, 100.0), unit=sc.units.us)
histo = sc.rebin(histo, Dim.Tof, edges)

#### Rebin  `EventWorkspace` preserving events

In [ ]:
mantid.Rebin(InputWorkspace='events', OutputWorkspace='events', Params='0,100,20000', PreserveEvents=True)

Equivalent in scipp:

In [ ]:
edges = sc.Variable([Dim.Tof], values=np.arange(0.0, 20000.0, 100.0), unit=sc.units.us)
events.coords[Dim.Tof] = edges

#### Rebin  `EventWorkspace` into `Workspace2D`

In [ ]:
mantid.Rebin(InputWorkspace='events', OutputWorkspace='histo', Params='0,100,20000', PreserveEvents=False)

Equivalent in scipp:

In [ ]:
edges = sc.Variable([Dim.Tof], values=np.arange(0.0, 20000.0, 100.0), unit=sc.units.us)
histo = sc.histogram(events, edges)

### Neutron-scattering specific algorithms
#### ConvertUnits

In [ ]:
ConvertUnits(InputWorkspace='tof', OutputWorkspace='dspacing', Target='dSpacing')

Equivalent in scipp:

In [ ]:
dspacing = sc.neutron.convert(tof, Dim.Tof, Dim.DSpacing)